In [14]:
import pandas as pd
import unicodedata
import nltk

In [29]:
df_reddit = pd.read_csv('train_set.csv')

In [30]:
df_reddit[df_reddit['class']==0]

,text,class
0,Get two nice notebooks and write it down for e...,0
2,Hope you have a nice day,0
21,I don<U+0092>t think he<U+0092>s seen my message.,0
24,It<U+0092>s not uncommon for someone (especial...,0
26,It takes longer but it is much safer and well ...,0
...,...,...
12469,OP is a girl...,0
12470,He'll find out eventually.,0
12473,I don't understand your title.,0
12474,Insulting me now eh?,0


In [31]:
df_reddit[df_reddit['class']==1]

,text,class
1,I<U+0092>m sobbing reading this thread!,1
3,My wife came in when I was around half way thr...,1
4,I am crying a lot of happy tears right now.,1
5,Some asshole is cutting onions around me right...,1
6,He<U+0092>s my father in every sense of the wo...,1
...,...,...
12479,"Let the cops help them, crooks or not.",1
12480,That's wild and a hell of a close call.,1
12482,I got into her line because she'd started work...,1
12483,We've been friends for about a year or so.,1


In [32]:
def convert_unicode_to_character(text):
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')


In [33]:
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [34]:
from nltk.tokenize import word_tokenize

In [35]:
df_reddit['text'].apply(word_tokenize)

0        [Get, two, nice, notebooks, and, write, it, do...
1        [I, <, U+0092, >, m, sobbing, reading, this, t...
2                          [Hope, you, have, a, nice, day]
3        [My, wife, came, in, when, I, was, around, hal...
4        [I, am, crying, a, lot, of, happy, tears, righ...
                               ...                        
12480    [That, 's, wild, and, a, hell, of, a, close, c...
12481    [That, 's, pretty, much, what, my, wife, said, .]
12482    [I, got, into, her, line, because, she, 'd, st...
12483    [We, 've, been, friends, for, about, a, year, ...
12484    [As, for, how, we, got, together, ,, I, was, t...
Name: text, Length: 12485, dtype: object

In [36]:
def convert_unicode(text):
    return text.replace('<U+0092>', "'")

In [37]:
df_reddit['text'] = df_reddit['text'].apply(convert_unicode)

In [38]:
df_reddit

,text,class
0,Get two nice notebooks and write it down for e...,0
1,I'm sobbing reading this thread!,1
2,Hope you have a nice day,0
3,My wife came in when I was around half way thr...,1
4,I am crying a lot of happy tears right now.,1
...,...,...
12480,That's wild and a hell of a close call.,1
12481,That's pretty much what my wife said.,0
12482,I got into her line because she'd started work...,1
12483,We've been friends for about a year or so.,1


In [62]:
import re
def print_unicode_text(text):
    pattern = r'(<.*?>)'
    res = re.findall(pattern, text)
    if len(res) > 0:
        # for item in res:
        #     text.replace(item, '')
        #     unicode_value = int(item[3:-1], 16)  # Extract the hex value and convert it to an integer
        #     unicode_character = chr(unicode_value)
        #     print(unicode_character, ' ')
        print(res)
        
    return text

def remove_unicode_text(text):
    pattern = r'(<.*?>)'
    res = re.findall(pattern, text)
    if len(res) > 0:
        for item in res:
            text = text.replace(item, '')
            # print(type(item))
        #     unicode_value = int(item[3:-1], 16)  # Extract the hex value and convert it to an integer
        #     unicode_character = chr(unicode_value)
        #     print(unicode_character, ' ')
        # print(text)
        
    return text

df_reddit['text'] = df_reddit['text'].apply(remove_unicode_text)

In [63]:
df_reddit['text'].apply(print_unicode_text)

0        Get two nice notebooks and write it down for e...
1                         I'm sobbing reading this thread!
2                                 Hope you have a nice day
3        My wife came in when I was around half way thr...
4              I am crying a lot of happy tears right now.
                               ...                        
12480              That's wild and a hell of a close call.
12481                That's pretty much what my wife said.
12482    I got into her line because she'd started work...
12483           We've been friends for about a year or so.
12484    As for how we got together, I was taking care ...
Name: text, Length: 12485, dtype: object

In [91]:
def replace_in_df(df, regex, text):
    def func(row):
        return re.sub(regex, text, row["text"])
    df['text'] = df.apply(func, axis=1).tolist()
    return df

def clean_tweets(df, min_length=8):
    # delete the entire tweet (1/2)
    df = replace_in_df(df, r".*RT @.*", "") # retweets
    df = replace_in_df(df, r".*\[\[.*", "") # placeholders
    # delete only a part of the tweet
    df = replace_in_df(df, r"http(s?)://\S+", "") # links
    df = replace_in_df(df, r"@\S+", "") # mentions
    df = replace_in_df(df, r"#\S+", "") # hashtags
    df = replace_in_df(df, r"\n", " ") # newline
    df = replace_in_df(df, r"  +", " ") # multiple spaces
    df = replace_in_df(df, r"^ ", "") # space at the begin of line
    df = replace_in_df(df, r" $", "") # space at the end of line
    df = replace_in_df(df, r"(&amp;)", "&")
    # delete the entire tweet (2/2)
    # df = replace_if_lang(df, "") # texts not in English
    df = df[df['text'].map(len) >= min_length] # short text
    return df.drop_duplicates()

In [69]:
df_reddit = clean_tweets(df_reddit)

In [70]:
df_reddit

,text,class
0,Get two nice notebooks and write it down for e...,0
1,I'm sobbing reading this thread!,1
2,Hope you have a nice day,0
3,My wife came in when I was around half way thr...,1
4,I am crying a lot of happy tears right now.,1
...,...,...
12480,That's wild and a hell of a close call.,1
12481,That's pretty much what my wife said.,0
12482,I got into her line because she'd started work...,1
12483,We've been friends for about a year or so.,1


In [71]:
df_sens = df_reddit[df_reddit['class']==1].drop('class', axis=1)

In [72]:
df_sens

,text
1,I'm sobbing reading this thread!
3,My wife came in when I was around half way thr...
4,I am crying a lot of happy tears right now.
5,Some asshole is cutting onions around me right...
6,He's my father in every sense of the word but ...
...,...
12479,"Let the cops help them, crooks or not."
12480,That's wild and a hell of a close call.
12482,I got into her line because she'd started work...
12483,We've been friends for about a year or so.


In [73]:
df_ns = df_reddit[df_reddit['class']==0].drop('class', axis=1)

In [74]:
df_ns

,text
0,Get two nice notebooks and write it down for e...
2,Hope you have a nice day
21,I don't think he's seen my message.
24,It's not uncommon for someone (especially wome...
26,It takes longer but it is much safer and well ...
...,...
12469,OP is a girl...
12470,He'll find out eventually.
12473,I don't understand your title.
12474,Insulting me now eh?


In [89]:
df_corporate = pd.read_csv('Corporate-messaging-DFE.csv', encoding = "ISO-8859-1")
df_corporate = df_corporate[['text']]

In [92]:
df_corporate = clean_tweets(df_corporate)
df_corporate

,text
0,Barclays CEO stresses the importance of regula...
1,Barclays announces result of Rights Issue
2,Barclays publishes its prospectus for its å£5....
3,Barclays Group Finance Director Chris Lucas is...
4,Barclays announces that Irene McDermott Brown ...
...,...
3109,We're working hard to do all we can to promote...
3111,Wish cities were more child-friendly? See how ...
3114,Wouldn't it be great if your oven recognised y...
3116,"Yo-Jelly, Danone new brand in South Africa : t..."


In [94]:
!pip install scikit-learn

from sklearn.model_selection import train_test_split


  Using cached scikit_learn-1.2.2-cp310-cp310-win_amd64.whl (8.3 MB)
  Using cached scipy-1.10.1-cp310-cp310-win_amd64.whl (42.5 MB)
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)


In [95]:
df_sens_train, df_sens_val = train_test_split(df_sens, test_size=0.2, random_state=42)

In [96]:
df_ns_train, df_ns_val = train_test_split(df_ns, test_size=0.2, random_state=42)

In [97]:
df_corporate_train, df_corporate_val = train_test_split(df_corporate, test_size=0.2, random_state=42)

In [99]:
df_sens_train, df_sens_val, df_ns_train, df_ns_val, df_corporate_train, df_corporate_val

(                                                    text
 7145   If your diet is healthy, then you will feel be...
 619                       Don't take your lead from her.
 11457         Thank you so much for sharing your story!!
 4707                          Sounds like you got hosed.
 4545   It sound to me like you want to live and be ha...
 ...                                                  ...
 8197   Kids always, always, always get their behavior...
 8447           It's something I've always thought about.
 1269   There is no upside to having him be a part of ...
 12143  The last I heard was that the parents bodies w...
 11577              but ill get on steam sometime haha :)
 
 [6288 rows x 1 columns],
                                                     text
 11299  And she found the people who complimented her ...
 2092   Just... don't take on blame for something you ...
 1494   Research indicates that memories from teenage ...
 3068                     So get yourself in

In [100]:
!pip install nlpaug


                                              0.0/410.5 kB ? eta -:--:--
     --                                       30.7/410.5 kB ? eta -:--:--
     --                                       30.7/410.5 kB ? eta -:--:--
     ---------                            112.6/410.5 kB 939.4 kB/s eta 0:00:01
     ---------                            112.6/410.5 kB 939.4 kB/s eta 0:00:01
     -----------------                    204.8/410.5 kB 892.5 kB/s eta 0:00:01
     -------------------                  225.3/410.5 kB 919.0 kB/s eta 0:00:01
     ----------------------               256.0/410.5 kB 830.3 kB/s eta 0:00:01
     -----------------------------        337.9/410.5 kB 999.0 kB/s eta 0:00:01
     -----------------------------        337.9/410.5 kB 999.0 kB/s eta 0:00:01
     ------------------------------------ 410.5/410.5 kB 950.8 kB/s eta 0:00:00


In [101]:
import nlpaug.augmenter.word as naw


In [127]:
text = "If and when you do meet that person, be prepared."
aug = naw.SynonymAug()
aug.augment(text)

['If and when you do meet that soul, be prepared.']

In [131]:
df_corporate_train.to_csv('df_corporate_train.csv', index=False)

In [132]:
df_whisper = pd.read_csv('whisper.csv')

In [133]:
df_whisper = df_whisper.drop('class', axis=1)

In [134]:
df_whisper

,text
0,My mom just sold my fucking bed without tellin...
1,Me and this guy had a full heart to heart conv...
2,Who am I even anymore?
3,I'm a post man. N would love a lonely houswife...
4,"I've been getting butterflies when we talk, an..."
...,...
3018167,because of tumblr girls I wish I wasn't my race
3018168,Still looking to find that first attractive gi...
3018169,Just dropped my phone on my face three times w...
3018170,"I never really appreciated my mom until, as a ..."


In [137]:
combined_df = pd.concat([df_whisper, df_sens, df_ns], ignore_index=True)


In [140]:
combined_df.to_csv('mixed.csv', index=False)

In [142]:
df_corporate.to_csv('df_corporate.csv', index=False)

In [143]:
def clean_text_alphanumeric(text):
    # Use regular expression to match non-English characters and non-numbers
    cleaned_text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    return cleaned_text

In [144]:
df_corporate_gen = pd.read_csv('df_corporate_gen.csv')

In [145]:
df_ns_gen = pd.read_csv('df_ns_gen.csv')

In [146]:
df_ns_combined = pd.concat([df_ns, df_ns_gen], ignore_index=True)

In [148]:
df_ns_combined.drop_duplicates()

,text
0,Get two nice notebooks and write it down for e...
1,Hope you have a nice day
2,I don't think he's seen my message.
3,It's not uncommon for someone (especially wome...
4,It takes longer but it is much safer and well ...
...,...
4620,OP is a girl...
4621,He'll find out eventually.
4622,I don't understand your title.
4623,Insulting me now eh?


In [149]:
df_ns_gen

,text
0,What he doesn't seem to understand is that The...
1,All part of the plan.
2,And thank you for your comment.
3,"Yes, and I would much rather do it than wait f..."
4,I even had a black widow living just outside m...
...,...
5370,That's true for humans too.
5371,But why is that making it difficult?
5372,And just remember: they've seen your CV and th...
5373,"It was black, blue, green, yellow for 2 weeks!"


In [150]:
df_ns_gen.drop_duplicates()

,text
0,What he doesn't seem to understand is that The...
1,All part of the plan.
2,And thank you for your comment.
3,"Yes, and I would much rather do it than wait f..."
4,I even had a black widow living just outside m...
...,...
5357,In moments of anger people think of hurting ot...
5361,It could be something new.
5363,I'm onto you guys ;)
5365,"If you haven't found your person yet, just enj..."


In [159]:
def augment_data(df) :   
    desired_samples = 10000

    # Create an empty list to store augmented samples
    augmented_samples = []

    # Initialize the augmentation object
    aug = naw.SynonymAug()

    # Augment the text samples
    while len(augmented_samples)+len(df) < desired_samples:
        for _, row in df.iterrows():
            text = row['text']

            # Augment the text multiple times until the desired number of samples is reached

            augmented_text = aug.augment(text)
            augmented_samples.append(augmented_text[0])

            # Break the loop if the desired number of samples is reached
            if len(augmented_samples)+len(df) >= desired_samples:
                break

    # Create a new DataFrame with augmented samples
    augmented_df = pd.DataFrame({'text': augmented_samples})

    # Concatenate the original DataFrame and the augmented DataFrame
    combined_df = pd.concat([df, augmented_df], ignore_index=True)

    # Save the combined DataFrame to a CSV file
    return combined_df

In [166]:
df_ns_final = augment_data(df_ns)
df_ns_final = df_ns_final.drop_duplicates()
while len(df_ns_final) != 10000:
    df_ns_final = augment_data(df_ns_final)
    df_ns_final = df_ns_final.drop_duplicates()

In [167]:
df_ns_final.drop_duplicates()

,text
0,Get two nice notebooks and write it down for e...
1,Hope you have a nice day
2,I don't think he's seen my message.
3,It's not uncommon for someone (especially wome...
4,It takes longer but it is much safer and well ...
...,...
9995,Aroused Trauma be the eccentric of psychic tra...
9996,Flummox two nice notebook and write it down fo...
9997,Bob hope you have a nice day
9998,I don ' t think he ' siemens project my subjec...


In [168]:
df_corporate_final = augment_data(df_corporate)
df_corporate_final = df_corporate_final.drop_duplicates()
while len(df_corporate_final) != 10000:
    df_corporate_final = augment_data(df_corporate_final)
    df_corporate_final = df_corporate_final.drop_duplicates()
df_corporate_final.drop_duplicates()

,text
0,Barclays CEO stresses the importance of regula...
1,Barclays announces result of Rights Issue
2,Barclays publishes its prospectus for its å£5....
3,Barclays Group Finance Director Chris Lucas is...
4,Barclays announces that Irene McDermott Brown ...
...,...
9995,Latest inquiry shows Gen Z values flexible inf...
9996,Barclays are proud sponsor of Legal Week Priva...
9997,Numbers in refuge  ÛÒ welcome to the weird an...
9998,Barclays CEO stresses the grandness of regulat...


In [197]:
len(df_sens.drop_duplicates())

7860

In [198]:
df_sens_final = df_sens
df_whisper_split = df_whisper.iloc[:2140]
df_sens_final = pd.concat([df_sens_final, df_whisper_split], ignore_index=True)
# while len(df_sens_final) != 10000:
#     for _, row in df_whisper.iterrows():
#         df_sens_final = pd.concat([df_sens_final, row], axis=0, ignore_index=True)
#         break

In [200]:
df_sens_final.drop_duplicates()

,text
0,I'm sobbing reading this thread!
1,My wife came in when I was around half way thr...
2,I am crying a lot of happy tears right now.
3,Some asshole is cutting onions around me right...
4,He's my father in every sense of the word but ...
...,...
9995,"stuck in work on such a nice day, cheer me up ..."
9996,I wanna have gay sex. Fit Gay dudes hmu
9997,I love girls that love girls it like living in...
9998,I hate that I have fallen for you.


In [209]:
df_sens_final['FriendsAndFamily'] = 1
df_sens_final['Professionals'] = 0
df_sens_final['RespectiveIndividuals'] = 0
df_sens_final['Strangers'] = 0


In [210]:
df_sens_final

,text,FriendsAndFamily,Professionals,RespectiveIndividuals,Strangers
0,I'm sobbing reading this thread!,1,0,0,0
1,My wife came in when I was around half way thr...,1,0,0,0
2,I am crying a lot of happy tears right now.,1,0,0,0
3,Some asshole is cutting onions around me right...,1,0,0,0
4,He's my father in every sense of the word but ...,1,0,0,0
...,...,...,...,...,...
9995,"stuck in work on such a nice day, cheer me up ...",1,0,0,0
9996,I wanna have gay sex. Fit Gay dudes hmu,1,0,0,0
9997,I love girls that love girls it like living in...,1,0,0,0
9998,I hate that I have fallen for you.,1,0,0,0


In [211]:
df_ns_final['FriendsAndFamily'] = 1
df_ns_final['Professionals'] = 0
df_ns_final['RespectiveIndividuals'] = 1
df_ns_final['Strangers'] = 1

In [212]:
df_ns_final

,text,FriendsAndFamily,Professionals,RespectiveIndividuals,Strangers
0,Get two nice notebooks and write it down for e...,1,0,1,1
1,Hope you have a nice day,1,0,1,1
2,I don't think he's seen my message.,1,0,1,1
3,It's not uncommon for someone (especially wome...,1,0,1,1
4,It takes longer but it is much safer and well ...,1,0,1,1
...,...,...,...,...,...
9995,Aroused Trauma be the eccentric of psychic tra...,1,0,1,1
9996,Flummox two nice notebook and write it down fo...,1,0,1,1
9997,Bob hope you have a nice day,1,0,1,1
9998,I don ' t think he ' siemens project my subjec...,1,0,1,1


In [213]:
df_corporate_final['FriendsAndFamily'] = 0
df_corporate_final['Professionals'] = 1
df_corporate_final['RespectiveIndividuals'] = 0
df_corporate_final['Strangers'] = 0

In [214]:
df_corporate_final

,text,FriendsAndFamily,Professionals,RespectiveIndividuals,Strangers
0,Barclays CEO stresses the importance of regula...,0,1,0,0
1,Barclays announces result of Rights Issue,0,1,0,0
2,Barclays publishes its prospectus for its å£5....,0,1,0,0
3,Barclays Group Finance Director Chris Lucas is...,0,1,0,0
4,Barclays announces that Irene McDermott Brown ...,0,1,0,0
...,...,...,...,...,...
9995,Latest inquiry shows Gen Z values flexible inf...,0,1,0,0
9996,Barclays are proud sponsor of Legal Week Priva...,0,1,0,0
9997,Numbers in refuge  ÛÒ welcome to the weird an...,0,1,0,0
9998,Barclays CEO stresses the grandness of regulat...,0,1,0,0


In [215]:
df_sens_final['text'] = df_sens_final['text'].apply(clean_text_alphanumeric)

In [216]:
df_ns_final['text'] = df_ns_final['text'].apply(clean_text_alphanumeric)
df_corporate_final['text'] = df_corporate_final['text'].apply(clean_text_alphanumeric)

In [217]:
df_sens_final,df_ns_final,df_corporate_final

(                                                   text  FriendsAndFamily   
 0                      I m sobbing reading this thread                  1  \
 1     My wife came in when I was around half way thr...                 1   
 2           I am crying a lot of happy tears right now                  1   
 3     Some asshole is cutting onions around me right...                 1   
 4     He s my father in every sense of the word but ...                 1   
 ...                                                 ...               ...   
 9995  stuck in work on such a nice day  cheer me up ...                 1   
 9996            I wanna have gay sex  Fit Gay dudes hmu                 1   
 9997  I love girls that love girls it like living in...                 1   
 9998                 I hate that I have fallen for you                  1   
 9999  When I was younger people made fun of my big b...                 1   
 
       Professionals  RespectiveIndividuals  Strangers  
 0   

In [218]:
df_sens_final_train, df_sens_final_val = train_test_split(df_sens_final, test_size=0.2, random_state=42)

In [219]:
df_ns_final_train, df_ns_final_val = train_test_split(df_ns_final, test_size=0.2, random_state=42)
df_corporate_final_train, df_corporate_final_val = train_test_split(df_corporate_final, test_size=0.2, random_state=42)

In [220]:
df_sens_final_train

,text,FriendsAndFamily,Professionals,RespectiveIndividuals,Strangers
9254,I hate having my driving license but no car,1,0,0,0
1561,The break up was horrific we were together for...,1,0,0,0
1670,Once you bring a child into the world you lose...,1,0,0,0
6087,Maybe random sights along the way,1,0,0,0
6669,A buddy of mine dropped a metal album recently...,1,0,0,0
...,...,...,...,...,...
5734,No I definitely argued with my wife about it ...,1,0,0,0
5191,The program even covers spending money meal...,1,0,0,0
5390,It s something I ve always thought about,1,0,0,0
860,There is no upside to having him be a part of ...,1,0,0,0


In [221]:
df_sens_final_val

,text,FriendsAndFamily,Professionals,RespectiveIndividuals,Strangers
6252,Distant a bit from parents a couple good frie...,1,0,0,0
4684,Have you tried going to therapy,1,0,0,0
1731,Now I m not saying you should make drastic ch...,1,0,0,0
4742,So I m a military widow and I know how much of...,1,0,0,0
4521,You got this for 2018,1,0,0,0
...,...,...,...,...,...
6412,Does she have a cool scar to go along with the...,1,0,0,0
8285,Hopefully I don t get arrested for leaving the...,1,0,0,0
7853,He loves her more than coffee but he would ne...,1,0,0,0
1095,It wasn t planned one thing led to another,1,0,0,0


In [222]:
df_ns_final_train

,text,FriendsAndFamily,Professionals,RespectiveIndividuals,Strangers
9254,Please get off your wyrd random bitter soapbox,1,0,1,1
1561,Recognizing that is helpful as you will be mor...,1,0,1,1
1670,Why on earth would we suddenly be aborting all...,1,0,1,1
6087,There be some hoods in Toronto such as Jane an...,1,0,1,1
6669,Nobody say you pitch enjoy your booty however...,1,0,1,1
...,...,...,...,...,...
5734,Less than ideal merely beggars can t person...,1,0,1,1
5191,In this world of which humans aren t convuls...,1,0,1,1
5390,Comfortably to acknowledge than later,1,0,1,1
860,They wonder Is this it,1,0,1,1


In [223]:
df_ns_final_val

,text,FriendsAndFamily,Professionals,RespectiveIndividuals,Strangers
6252,You know you remember you know merely you ...,1,0,1,1
4684,If the horseshoe fit though,1,0,1,1
1731,Also put tape on your webcam dude,1,0,1,1
4742,It wasn t easy but information technology w...,1,0,1,1
4521,In India not lot of people keep the lane wai...,1,0,1,1
...,...,...,...,...,...
6412,Oftentimes the process live challenge for ever...,1,0,1,1
8285,Ace was prepared for information technology,1,0,1,1
7853,Even if you can t find a write platter about...,1,0,1,1
1095,Show him he was wrong by being someone that in...,1,0,1,1


In [224]:
df_corporate_final_train

,text,FriendsAndFamily,Professionals,RespectiveIndividuals,Strangers
9254,We ve published our 2010 Creating Shared Tim...,0,1,0,0
1561,We re supporting coffee farmers in India as pa...,0,1,0,0
1670,Tests reconfirmed the safety of NAN H A 1 Gol...,0,1,0,0
6087,Prison term spent on electronic mail can be cu...,0,1,0,0
6669,investor yell coming to an end join us soon fo...,0,1,0,0
...,...,...,...,...,...
5734,Exclusive Investor want buyback spinoffs on ...,0,1,0,0
5191,Investor biggest concern focus on the develo...,0,1,0,0
5390,Srinivas Siripurapu appointed Head of Southeas...,0,1,0,0
860,Statement by Citi Chairman Richard D Parsons ...,0,1,0,0


In [225]:
df_corporate_final_val

,text,FriendsAndFamily,Professionals,RespectiveIndividuals,Strangers
6252,Empowering those w HIV to cultivate with the...,0,1,0,0
4684,As a loss leader in the handling of advanced k...,0,1,0,0
1731,Coffee farmer Cesar one of 147 000 working w...,0,1,0,0
4742,Kalamazoo MI nursing student s experience ...,0,1,0,0
4521,chief financial officer says portfolio directi...,0,1,0,0
...,...,...,...,...,...
6412,Want to improve your health Read astir the gr...,0,1,0,0
8285,Bank can buoy facilitate to halt human traffic...,0,1,0,0
7853,Barclays Riches be proud to sponsor Trace Catc...,0,1,0,0
1095,Corporate Responsibility at Merck Products d...,0,1,0,0


In [226]:
df_final_train = pd.concat([df_sens_final_train, df_ns_final_train, df_corporate_final_train])

In [227]:
df_final_train

,text,FriendsAndFamily,Professionals,RespectiveIndividuals,Strangers
9254,I hate having my driving license but no car,1,0,0,0
1561,The break up was horrific we were together for...,1,0,0,0
1670,Once you bring a child into the world you lose...,1,0,0,0
6087,Maybe random sights along the way,1,0,0,0
6669,A buddy of mine dropped a metal album recently...,1,0,0,0
...,...,...,...,...,...
5734,Exclusive Investor want buyback spinoffs on ...,0,1,0,0
5191,Investor biggest concern focus on the develo...,0,1,0,0
5390,Srinivas Siripurapu appointed Head of Southeas...,0,1,0,0
860,Statement by Citi Chairman Richard D Parsons ...,0,1,0,0


In [231]:
df_randomized_train = df_final_train.sample(frac=1, random_state=42).reset_index(drop=True)

In [232]:
df_randomized_train

,text,FriendsAndFamily,Professionals,RespectiveIndividuals,Strangers
0,gt And also have you ever became really clos...,1,0,0,0
1,Bristol Myers Squibb and Pfizer Receive Comp...,0,1,0,0
2,Our still has 13 days of safety,0,1,0,0
3,In more than than 1 200 be enter in activitie...,0,1,0,0
4,Pfizer Global Health Fellow Albert Dunne addre...,0,1,0,0
...,...,...,...,...,...
23995,A Banamex tradition 8 yrs 13 sea turtle camps...,0,1,0,0
23996,I miss my ex All day Every day,1,0,0,0
23997,I m looking forward to being like this when I ...,1,0,0,0
23998,Ar you happy with your current weight,1,0,1,1


In [234]:
df_final_val = pd.concat([df_sens_final_val, df_ns_final_val, df_corporate_final_val])

df_randomized_val = df_final_val.sample(frac=1, random_state=42).reset_index(drop=True)

In [235]:
df_randomized_val

,text,FriendsAndFamily,Professionals,RespectiveIndividuals,Strangers
0,No advice really I feel your pain though,1,0,0,0
1,Have a safe piazza to go to,1,0,1,1
2,I ve gone alone a couple of times to see some ...,1,0,0,0
3,Advantageously be a child or a click a person...,1,0,1,1
4,Josh Silver medal We Just Dumped a Bag of Mon...,0,1,0,0
...,...,...,...,...,...
5995,So skillful to get wind such a wholesome post,1,0,1,1
5996,Left your gift shopping to the last place inst...,0,1,0,0
5997,We re the 1 to sign up up to a European i...,0,1,0,0
5998,Deb Connor rebuilds the homes of typhoon survi...,0,1,0,0


In [236]:
df_randomized_train.to_csv('df_randomized_train.csv', index=False)
df_randomized_val.to_csv('df_randomized_val.csv', index=False)